# Semantic Kernel 

このコードサンプルでは、[Semantic Kernel](https://aka.ms/ai-agents-beginners/semantic-kernel) AI フレームワークを使用して基本的なエージェントを作成します。

このサンプルの目的は、さまざまなエージェントパターンを実装する際に、後続のコードサンプルで使用する手順を示すことです。

## 必要なPythonパッケージのインポート

In [1]:
import json
import os 

from typing import Annotated

from dotenv import load_dotenv

from IPython.display import display, HTML

from openai import AsyncOpenAI

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent
from semantic_kernel.functions import kernel_function

## クライアントの作成

このサンプルでは、LLMへのアクセスに[GitHub Models](https://aka.ms/ai-agents-beginners/github-models)を使用します。

`ai_model_id`は`gpt-4o-mini`として定義されています。GitHub Models マーケットプレイスで利用可能な別のモデルに変更して、異なる結果を確認してみてください。

GitHub Models の `base_url` に使用される `Azure Inference SDK` を使用するため、Semantic Kernel 内で `OpenAIChatCompletion` コネクターを使用します。Semantic Kernel を他のモデルプロバイダーで使用するための[利用可能なコネクター](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion)もあります。

In [2]:
import random   

# サンプル用のプラグインを定義

class DestinationsPlugin:
    """休暇のためのランダムな目的地のリスト"""

    def __init__(self):
        # 休暇の目的地のリスト
        self.destinations = [
            "バルセロナ、スペイン",
            "パリ、フランス",
            "ベルリン、ドイツ",
            "東京、日本",
            "シドニー、オーストラリア",
            "ニューヨーク、アメリカ",
            "カイロ、エジプト",
            "ケープタウン、南アフリカ",
            "リオデジャネイロ、ブラジル",
            "バリ、インドネシア"
        ]
        # 重複を避けるために最後の目的地を追跡
        self.last_destination = None

    @kernel_function(description="ランダムな休暇の目的地を提供します。")
    def get_random_destination(self) -> Annotated[str, "ランダムな休暇の目的地を返します。"]:
        # 利用可能な目的地を取得（可能であれば最後の目的地を除く）
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # ランダムに目的地を選択
        destination = random.choice(available_destinations)

        # 最後の目的地を更新
        self.last_destination = destination

        return destination

In [3]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# `ChatCompletionAgent` で使用される AI サービスを作成
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## エージェントの作成

以下で `TravelAgent` というエージェントを作成します。

この例では、非常にシンプルな指示を使用しています。これらの指示を変更して、エージェントがどのように異なる応答をするかを確認できます。

In [4]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="あなたは顧客のランダムな目的地での休暇計画を支援できる有用なAIエージェントです",
)

## エージェントの実行

`ChatHistory` を定義し、`system_message` を追加することで、エージェントを実行できます。先ほど定義した `AGENT_INSTRUCTIONS` を使用します。

これらが定義された後、ユーザーがエージェントに送信する内容である `user_inputs` を作成します。この場合、このメッセージを `日帰り旅行を計画して` に設定しています。

このメッセージを変更して、エージェントがどのように異なる応答をするかを確認してみてください。

In [6]:
user_inputs = [
    "月面旅行を計画してください。",
    "火星に行く計画も立てたいけどどうかな？",
]

async def main():
    thread: ChatHistoryAgentThread | None = None

    for user_input in user_inputs:
        html_output = (
            f"<div style='margin-bottom:10px'>"
            f"<div style='font-weight:bold'>ユーザー:</div>"
            f"<div style='margin-left:20px'>{user_input}</div></div>"
        )

        agent_name = None
        full_response: list[str] = []
        function_calls: list[str] = []

        # ストリーミング関数呼び出しを再構築するためのバッファ
        current_function_name = None
        argument_buffer = ""

        async for response in agent.invoke_stream(
            messages=user_input,
            thread=thread,
        ):
            thread = response.thread
            agent_name = response.name
            content_items = list(response.items)

            for item in content_items:
                if isinstance(item, FunctionCallContent):
                    if item.function_name:
                        current_function_name = item.function_name

                    # 引数を蓄積（チャンクでストリーミング）
                    if isinstance(item.arguments, str):
                        argument_buffer += item.arguments
                elif isinstance(item, FunctionResultContent):
                    # 結果を表示する前に保留中の関数呼び出しを終了
                    if current_function_name:
                        formatted_args = argument_buffer.strip()
                        try:
                            parsed_args = json.loads(formatted_args)
                            formatted_args = json.dumps(parsed_args)
                        except Exception:
                            pass  # 生文字列として残す

                        function_calls.append(f"関数呼び出し: {current_function_name}({formatted_args})")
                        current_function_name = None
                        argument_buffer = ""

                    function_calls.append(f"\n関数結果:\n\n{item.result}")
                elif isinstance(item, StreamingTextContent) and item.text:
                    full_response.append(item.text)

        if function_calls:
            html_output += (
                "<div style='margin-bottom:10px'>"
                "<details>"
                "<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>関数呼び出し（クリックして展開）</summary>"
                "<div style='margin:10px; padding:10px; background-color:#f8f8f8; "
                "border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;'>"
                f"{chr(10).join(function_calls)}"
                "</div></details></div>"
            )

        html_output += (
            "<div style='margin-bottom:20px'>"
            f"<div style='font-weight:bold'>{agent_name or 'アシスタント'}:</div>"
            f"<div style='margin-left:20px; white-space:pre-wrap'>{''.join(full_response)}</div></div><hr>"
        )

        display(HTML(html_output))

await main()